In [ ]:
# Imports
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt
import contextily as ctx
import osmnx as ox
import networkx as nx
from shapely.geometry import LineString, Point, Polygon, MultiPoint
from shapely.ops import nearest_points
from shapely.ops import unary_union
from shapely import speedups
from geopy.geocoders import Nominatim
from geopandas.tools import geocode
from geopy.extra.rate_limiter import RateLimiter
import datetime

%matplotlib inline

## 1.
Travel times

Functioning locations: 
- Yliopistonkatu 3 = Porthania
- Haartmaninkatu 8 = Meilahti
- Gustaf Hällströmin katu 2 = Kumpula
- Viikinkaari 11 = Viikki

In [ ]:
epsg=3067

In [ ]:
# Test addresses
#orig_address = 'Radiokuja 6, 00240, Helsinki, Finland'
#dest_address = 'Väinö Auerin katu 13, 00560, Helsinki, Finland'

# Test addresses2 and transportation method
#orig_address = 'Yliopistonkatu 3, 00100, Helsinki, Finland'
#dest_address = 'Haartmaninkatu 8, 00290, Helsinki, Finland'
#transport_method = 'walk'

# Addresses and transportation method with user input
orig_address = str(input('Origin address:')+', Helsinki, Finland')
dest_address = str(input('Destination address:')+', Helsinki, Finland')
transport_method = str(input('transportation method (walk, bike, public, or car):'))

# Geocode addresses
orig_point = geocode(orig_address, provider='nominatim', user_agent='autogis_EH', timeout=10)
dest_point = geocode(dest_address, provider='nominatim', user_agent='autogis_EH', timeout=10)

# Project to ETRS-TM35FIN
orig_point = orig_point.to_crs(epsg=epsg)
dest_point = dest_point.to_crs(epsg=epsg)

In [ ]:
# Reading in the YKR grid
fp_grid = r'data/YKR/MetropAccess_YKR_grid_EurefFIN.shp'
grid = gpd.read_file(fp_grid)

# Checking projections
if grid.crs == orig_point.crs == dest_point.crs:
    print('data is correctly projected')
else:
    print('projections do not match')

In [ ]:
# Defining centroids for the YKR grid

# Dataframe for centroids
grid_centroids = gpd.GeoDataFrame()

# Defining a centroid for every grid cell
grid_centroids['geometry'] = gpd.points_from_xy(grid.x, grid.y)

# Keeping the YKR_ID for data selection
grid_centroids['YKR_ID'] = grid['YKR_ID']

# Results
grid_centroids.head()

In [ ]:
# Function for getting the YKR_ID value of the nearest cell centroid
def get_nearest_values(row, other_gdf, point_column='geometry', value_column='geometry'):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf['geometry'].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf['geometry'] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [ ]:
# Applying the function 

# Finding closest centroid to orig_point 
orig_point['YKR_ID'] = orig_point.apply(
    get_nearest_values,
    other_gdf=grid_centroids,
    point_column='geometry',
    value_column='YKR_ID',
    axis=1
    )

# Finding closest centroid to dest_point
dest_point['YKR_ID'] = dest_point.apply(
    get_nearest_values,
    other_gdf=grid_centroids,
    point_column='geometry',
    value_column='YKR_ID',
    axis=1
    )

# Assigning YKR_IDs to variables
#orig_YKR = orig_point.loc[0, 'YKR_ID']
#dest_YKR = dest_point.loc[0, 'YKR_ID']

orig_YKR = orig_point['YKR_ID'].iloc[0]
dest_YKR = dest_point['YKR_ID'].iloc[0]
print(orig_YKR, dest_YKR)

In [ ]:
# Select apprpriate travel time data based on dest_point's YKR_ID
fp_tt = r'data/traveltimes/travel_times_to_ '+str(dest_YKR)+'.txt'

# Read in data
tt_data = pd.read_csv(fp_tt, sep=';', na_values=['-1'])

# Merge travel time data to YKR grid and drop na values
tt_grid = grid.merge(tt_data, on=tt_data['from_id'])
tt_grid = tt_grid.dropna()
tt_grid.columns

In [ ]:
# Get travel time from orig_point to dest_point
time_cell = tt_grid.loc[tt_grid['YKR_ID']==orig_YKR]

if transport_method == 'walk':
    column = 'walk_t'
if transport_method == 'bike':
    column = 'bike_s_t'
if transport_method == 'public':
    column = 'pt_r_t'
if transport_method == 'car':
    column = 'car_r_t'

travel_time = time_cell[column].iloc[0]
travel_time

In [ ]:
# ETA
time_now = datetime.datetime.utcnow()+datetime.timedelta(hours=2)

eta = time_now + datetime.timedelta(minutes=travel_time)
print('Time now: '+
      str(round(time_now.hour, 2))+
      ':'+
      str(round(time_now.minute, 2))+
      '\n\nEstimated time of arrival: '+
      str(eta.hour)+
      ':'+
      str(eta.minute)
     )

## 2.
Path

In [ ]:
# orig and dest points to wgs
orig_point_wgs = orig_point.to_crs(epsg=4326)
dest_point_wgs = dest_point.to_crs(epsg=4326)

# Combine the dest and orig points
extent_points = orig_point_wgs.append(dest_point_wgs)

# Create a convex hull
graph_extent = extent_points.unary_union.convex_hull

# Conditional buffer to limit network size for walking and cycling
if transport_method == 'walk':
    buffer = 0.01
if transport_method == 'bike':
    buffer = 0.01
if transport_method == 'public':
    buffer = 0.03
if transport_method == 'car':
    buffer = 0.03

graph_extent = graph_extent.buffer(buffer)

# Check results
graph_extent

In [ ]:
# Select OSM network type conditionally
if transport_method == 'walk':
    network = 'walk'
if transport_method == 'bike':
    network = 'bike'
if transport_method == 'public':
    network = 'drive'
if transport_method == 'car':
    network = 'drive'

# Get graph
graph = ox.graph_from_polygon(graph_extent, network_type=network)

In [ ]:
# Project graph to UTM zone 35
graph_utm = ox.project_graph(graph)
nodes_utm, edges_utm = ox.graph_to_gdfs(graph_utm, nodes=True, edges=True)

# Project points to UTM zone 35
orig_point_utm = orig_point.to_crs(edges_utm.crs)
dest_point_utm = dest_point.to_crs(edges_utm.crs)

In [ ]:
edges_utm.plot()

In [ ]:
# Creating GDF with a line column for the route
routes = gpd.GeoDataFrame()
routes['geometry'] = None

# Select origin y and x
orig_yx = (orig_point_utm['geometry'].iloc[0].y, orig_point_utm['geometry'].iloc[0].x)
# Get node id
orig_node_id = ox.get_nearest_node(graph_utm, orig_yx, method='euclidean')
# Get node
orig_node = nodes_utm.loc[orig_node_id]

# Select destination y and x
dest_yx = (dest_point_utm['geometry'].iloc[0].y, dest_point_utm['geometry'].iloc[0].x)
# Get node id
dest_node_id = ox.get_nearest_node(graph_utm, dest_yx, method='euclidean')
# Get node
dest_node = nodes_utm.loc[dest_node_id]

# Only calculate route when the start and destination node IDs are different
if dest_node_id != orig_node_id:
    # Get shortest path
    route = nx.shortest_path(graph_utm, source=orig_node_id, target=dest_node_id, weight='length')
    # Get route nodes
    route_nodes = nodes_utm.loc[route]
    # Construct route line
    route_line = LineString(list(route_nodes.geometry.values))
    # Add line to lines column
    routes.at[0, 'geometry'] = route_line

Tests

In [ ]:
# Cells to gdf
orig_cell = gpd.GeoDataFrame()
dest_cell = gpd.GeoDataFrame()

orig_cell = grid.loc[grid['YKR_ID']==orig_YKR]
dest_cell = grid.loc[grid['YKR_ID']==dest_YKR]

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))

#grid.plot(ax=ax, facecolor=[0.9,0.9,0.9], edgecolor=[0.7,0.7,0.7])
#tt_grid.plot(ax=ax, column='pt_r_t', cmap='RdYlBu')

#orig_cell.plot(ax=ax, facecolor=[0.9,0.8,0])
#dest_cell.plot(ax=ax, facecolor=[0,0.7,0.7])
orig_point.plot(ax=ax, color=[0,0,1])
dest_point.plot(ax=ax, color=[1,0,0])

edges_utm.plot(ax=ax, color=[0.8,0.8,0.8])
routes.plot(ax=ax)
#grid_centroids.plot(ax=ax, color=[0,0,0], markersize=1)